# 1 类别数据

In [2]:
# # 1 Background and Motivation（背景和动力）
# 
# 表格中的列克可能会有重复的部分。我们可以用unique和value_counts，从一个数组从提取不同的值，并计算频度：

In [3]:
import numpy as np
import pandas as pd

In [4]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [5]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [6]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [7]:
# 对于不同的类型数据值，一个更好的方法是用维度表（dimension table）来表示，然后用整数键（integer keys）来指代维度表：

In [8]:
values = pd.Series([0, 1, 0, 0] * 2)
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [9]:
dim = pd.Series(['apple', 'orange'])
dim

0     apple
1    orange
dtype: object

In [10]:
# 用take方法来重新存储原始的，由字符串构成的Series：

In [11]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [12]:
# 这种用整数表示的方法叫做类别（categorical）或字典编码（dictionary-encoded）表示法。表示不同类别值的数组，被称作类别，字典，或层级。本书中我们将使用类别（categorical and categories）来称呼。表示类别的整数值被叫做，类别编码（category code），或编码（code）。
# 
# # 2 Categorical Type in pandas（pandas中的Categorical类型）
# 
# pandas中有一个Categorical类型，是用来保存那些基于整数的类别型数据。考虑下面的例子：

In [13]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [14]:
N = len(fruits)

In [15]:
N

8

In [16]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,4,3.309550
1,1,orange,13,0.240087
2,2,apple,14,2.111382
3,3,apple,7,2.362748
4,4,apple,7,1.455337
5,5,orange,10,0.394405
6,6,apple,8,0.934908
7,7,apple,5,1.599654


In [17]:
# 这里，df['fruit']是一个python的字符串对象。我们将其转换为类型对象：

In [18]:
fruits_cat = df['fruit'].astype('category')
fruits_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [19]:
# fruits_cat的值并不是一个numpy数组，而是一个pandas.Categorical实例：

In [20]:
c = fruits_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

In [21]:
# 这个Categorical对象有categories和codes属性：

In [22]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [23]:
# 可以把转换的结果变为DataFrame列：

In [24]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [25]:
# 也可以直接把其他的python序列变为pandas.Categorical类型：

In [26]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [27]:
# 如果已经得到了分类编码数据（categorical encoded data），我们可以使用from_codes构造器：

In [28]:
categories = ['foo', 'bar', 'baz']

In [29]:
codes = [0, 1, 2, 0, 0, 1]

In [30]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [31]:
# 除非明确指定，非常默认类别没有特定的顺序。所以，取决于输入的数据，categories数组可能有不同的顺序。当使用from_codes或其他一些构造器的时候，我们可以指定类别的顺序：

In [32]:
ordered_cat = pd.Categorical.from_codes(codes, categories, 
                                        ordered=True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [33]:
# 输出的结果中，`[foo < bar < baz]`表示foo在bar之间，以此类推。一个没有顺序的类型实例（unordered categorical instance）可以通过as_ordered来排序：

In [34]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [35]:
# 最后一点需要注意的，类型数据没必要一定是字符串，它可以是任何不可变的值类型
# （any immutable value types）。
# 
# # 3 Computations with Categoricals（类型计算）
# 
# Categorical类型和其他类型差不多，不过对于某些函数，比如groupby函数，在Categorical数据上会有更好的效果。很多函数可以利用ordered标记。
# 
# 假设有一些随机的数字，用pandas.quct进行分箱（binning）。得到的类型是pandas.Categorical；虽然之前用到过pandas.cut，但是没有具体介绍里面的细节：

In [36]:
np.random.seed(12345)

In [37]:
draws = np.random.randn(1000)

In [38]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [39]:
# 计算分箱后的分位数：

In [40]:
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [41]:
# 具体分位数并不如季度的名字直观，我们直接在qcut中设定labels：

In [42]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [43]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [44]:
# bins caetegorical并没有包含边界星系，我们可以用groupby来提取：

In [45]:
bins = pd.Series(bins, name='quartile')

In [46]:
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [47]:
# quartile列包含了原始的类别信息，包含bins中的顺序：

In [48]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [49]:
# ### Better performance with categoricals （使用categoricals得到更好的效果）
# 
# 使用categorical能让效果提高。如果一个DataFrame的列是categorical类型，使用的时候会减少很多内存的使用。假设我们有一个一千万的元素和一个类别：


In [50]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [51]:
# 把labels变为categorical：

In [52]:
categories = labels.astype('category')

In [53]:
# 可以看到labels会比categories使用更多的内存：

In [54]:
labels.memory_usage()

80000080

In [55]:
categories.memory_usage()

10000272

In [56]:
# 当然，转换成category也是要消耗计算的，不过这种消耗是一次性的：

In [57]:
get_ipython().run_line_magic('time', "_ = labels.astype('category')")

Wall time: 576 ms


In [58]:
# 在categories上使用groupby会非常快，因为用的是基于整数的编码，而不是由字符串组成的数组。
# 
# # 4 Categorical Methods（类别方法）
# 
# 如果是包含categorical数据的Series数据，有和Series.str类似的一些比较特殊的方法。对于访问categories和code很方便：

In [59]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

In [60]:
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [62]:
# 属性cat可以访问categorical方法：
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [63]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [64]:
# 假设我们知道实际的类别超过了当前观测到的四个类别，那么我们可以使用set_categories方法来扩展：

In [65]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [66]:
# 数据本身似乎没有改变，不过在对其进行操作的时候会反应出来。例如，value_counts：

cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [67]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [68]:
# 在大型数据集，categoricals经常用来作为省内存和提高效果的工具。在对一个很大的DataFrame或Series进行过滤后，很多类型可能不会出现在数据中。我们用remove_unused_categories方法来除去没有观测到的类别：

In [69]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [70]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

In [71]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [73]:
# 在第七章也介绍过，pandas.get_dummies函数会把一维的类型数据变为包含哑变量的DataFrame：

pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


# 2高级Groupby用法

In [74]:
# 在第十章里，使用apply方法在组上进行转换操作的。还有一个内建的方法叫transform，和apply相同，但是在一些函数的用法上有一些限制：
# 
# - 可以产生一个标量，将数据广播（broadcast）到与组一样的形状（这里的broadcast可以理解为改变数据形状的方法，感兴趣的可以直接搜索 numpy broadcast）
# - 可以产生一个和输入的组一样形状的对象
# - 不能对输入进行改变
# 
# 举个例子：

In [75]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [77]:
# 通过key来计算组的平均值：

g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [78]:
# 假设我们想要产生一个和df['value']一样大小的Series，不过要用key分组后的平均值来替换。我们可以把函数lambda x: x.mean()给transform：

g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [81]:
# 对于内建的聚合函数，我们可以传入一个字符串别名，就像使用groupby agg方法的时候一样：

In [82]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [80]:
# 就像apply，transform能用那些返回Series的函数，但是结果的大小和输入的必须一样。例如，我们通过一个lambda函数令每个小组都乘2：

In [83]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [84]:
# 一个更复杂的例子，我们可以按降序来计算每一个组：

In [85]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [86]:
# 考虑一个包含简单聚合的分组转换函数：

In [87]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [88]:
# 使用transform或apply，都能得到一样的结果：

In [89]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [90]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [91]:
# 内建的聚合函数，比如mean, sum经常比一般的apply函数要快。而是用transform的话，会更快一些。这就需要我们使用无包装的组操作（upwrapped group operation）：

In [92]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [93]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [94]:
# 一个无包装的组操作可能会涉及多个组聚合操作，不过向量化操作会胜过这种操作。
# 
# # 2 Grouped Time Resampling（分组时间重采样）
# 
# 对于时间序列数据，resample方法是一个基于时间的组操作。这里有一个样本表格： 


In [96]:
N = 15
times = pd.date_range('2019-07-11 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times, 'value': np.arange(N)})
df

,time,value
0,2019-07-11 00:00:00,0
1,2019-07-11 00:01:00,1
2,2019-07-11 00:02:00,2
3,2019-07-11 00:03:00,3
4,2019-07-11 00:04:00,4
5,2019-07-11 00:05:00,5
6,2019-07-11 00:06:00,6
7,2019-07-11 00:07:00,7
8,2019-07-11 00:08:00,8
9,2019-07-11 00:09:00,9


In [97]:
# 我们用time索引，然后重采样：

In [98]:
df.set_index('time').resample('5min').count()

,value
time,
2019-07-11 00:00:00,5
2019-07-11 00:05:00,5
2019-07-11 00:10:00,5


In [99]:
# 假设一个DataFrame包含多个时间序列，用多一个key列来表示：

In [100]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N), 
                    'value': np.arange(N * 3.)})
df2[:7]

,time,key,value
0,2019-07-11 00:00:00,a,0.0
1,2019-07-11 00:00:00,b,1.0
2,2019-07-11 00:00:00,c,2.0
3,2019-07-11 00:01:00,a,3.0
4,2019-07-11 00:01:00,b,4.0
5,2019-07-11 00:01:00,c,5.0
6,2019-07-11 00:02:00,a,6.0


In [101]:
# 想要对key列的值做重采样，我们引入pandas.TimeGrouper对象：

In [102]:
time_key = pd.TimeGrouper('5min')

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """Entry point for launching an IPython kernel.


In [103]:
# 然后设置time为索引，对key和time_key做分组，然后聚合：

In [104]:
resampled = (df2.set_index('time')
             .groupby(['key', time_key])
             .sum())
resampled

value
key time                      
a   2019-07-11 00:00:00   30.0
    2019-07-11 00:05:00  105.0
    2019-07-11 00:10:00  180.0
b   2019-07-11 00:00:00   35.0
    2019-07-11 00:05:00  110.0
    2019-07-11 00:10:00  185.0
c   2019-07-11 00:00:00   40.0
    2019-07-11 00:05:00  115.0
    2019-07-11 00:10:00  190.0

In [105]:
resampled.reset_index()

,key,time,value
0,a,2019-07-11 00:00:00,30.0
1,a,2019-07-11 00:05:00,105.0
2,a,2019-07-11 00:10:00,180.0
3,b,2019-07-11 00:00:00,35.0
4,b,2019-07-11 00:05:00,110.0
5,b,2019-07-11 00:10:00,185.0
6,c,2019-07-11 00:00:00,40.0
7,c,2019-07-11 00:05:00,115.0
8,c,2019-07-11 00:10:00,190.0


In [106]:
# 使用TimeGrouper的一个限制是时间必须是Series或DataFrame的索引才行。